In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import VECM, select_order, select_coint_rank, coint_johansen
from time_series_utils import TSA

In [2]:
start_date = pd.Timestamp("2000-01-01")
end_date = pd.Timestamp("2019-12-31")

In [3]:
# "SPY": SPDR S&P 500 ETF Trust
# "IWM": iShares Russell 2000 ETF 
sp500_df = yf.download("SPY", start=start_date, end=end_date)
russell2000_df = yf.download("IWM", start=start_date, end=end_date)
display(sp500_df.head())
display(russell2000_df.head())
sp500_df.shape[0], russell2000_df.shape[0]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03 00:00:00-05:00,148.25000,148.25000,143.875000,145.4375,95.308823,8164300
2000-01-04 00:00:00-05:00,143.53125,144.06250,139.640625,139.7500,91.581657,8089800
2000-01-05 00:00:00-05:00,139.93750,141.53125,137.250000,140.0000,91.745483,12177900
2000-01-06 00:00:00-05:00,139.62500,141.50000,137.750000,137.7500,90.271019,6227200
2000-01-07 00:00:00-05:00,140.31250,145.75000,140.062500,145.7500,95.513580,8066500


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-05-26 00:00:00-04:00,45.531250,45.71875,45.312500,45.718750,33.982517,74800
2000-05-30 00:00:00-04:00,46.375000,47.40625,46.375000,47.406250,35.236824,57600
2000-05-31 00:00:00-04:00,47.562500,48.18750,47.562500,47.578125,35.364571,36000
2000-06-01 00:00:00-04:00,48.554688,48.65625,48.554688,48.656250,36.165947,7000
2000-06-02 00:00:00-04:00,50.859375,51.18750,50.859375,51.187500,38.047405,29400


(5030, 4929)

In [4]:
sp500 = sp500_df["Adj Close"]
russell2000 = russell2000_df["Adj Close"]
pair_df = pd.concat([sp500, russell2000], axis=1, join="inner")
pair_df.columns = ["sp500", "russell2000"]
pair_df

,sp500,russell2000
Date,,
2000-05-26 00:00:00-04:00,90.664665,33.982517
2000-05-30 00:00:00-04:00,93.621124,35.236824
2000-05-31 00:00:00-04:00,93.826431,35.364571
2000-06-01 00:00:00-04:00,95.468895,36.165947
2000-06-02 00:00:00-04:00,97.131897,38.047405
...,...,...
2019-12-23 00:00:00-05:00,306.215424,160.106964
2019-12-24 00:00:00-05:00,306.224945,160.472763
2019-12-26 00:00:00-05:00,307.855042,160.472763


In [5]:
train_df = pair_df.loc[pair_df.index < "2017-01-01"].reset_index(drop=True)
test_df = pair_df.loc[pair_df.index >= "2017-01-01"].reset_index(drop=True)

In [6]:
# difference sequence
d_train_df = train_df.diff().dropna()
d_test_df = test_df.diff().dropna()

# Stationarity Test

In [7]:
sp500_tsa = TSA(train_df.sp500.values)
ADF_stats, ADF_p_value, best_lag, terms = sp500_tsa.ADF_test_complete()
ADF_stats, ADF_p_value, best_lag, terms

(1.9040104428743452, 0.9873091156022337, 5, 'n')

In [8]:
russell2000_tsa = TSA(train_df.russell2000.values)
ADF_stats, ADF_p_value, best_lag, terms = russell2000_tsa.ADF_test_complete()
ADF_stats, ADF_p_value, best_lag, terms

(1.7672413821531539, 0.9820028581398815, 5, 'n')

Both sp500 and russell2000 are unstationary.

In [9]:
d_sp500_tsa = TSA(d_train_df.sp500.values)
ADF_stats, ADF_p_value, best_lag, terms = d_sp500_tsa.ADF_test_complete()
ADF_stats, ADF_p_value, best_lag, terms

(-28.466035776077746, 0.0, 5, 'n')

In [10]:
d_russell2000_tsa = TSA(d_train_df.russell2000.values)
ADF_stats, ADF_p_value, best_lag, terms = d_russell2000_tsa.ADF_test_complete()
ADF_stats, ADF_p_value, best_lag, terms

(-31.00067670562371, 0.0, 4, 'n')

Both difference of sp500 sequence and difference of russell2000 sequence are stationary.

# Cointegration Test

## ADF Test

In [11]:
ols = sm.OLS(train_df.sp500.values, train_df.russell2000.values)
est = ols.fit()
residue = est.resid
residue_tsa = TSA(residue)
ADF_stats, ADF_p_value, best_lag, terms = residue_tsa.ADF_test_complete()
ADF_stats, ADF_p_value, best_lag, terms

(-3.2068671175071555, 0.0013460413353862664, 4, 'n')

Residue is stationary. So sp500 and russell2000 are cointegrated.

## Johansen Test

In [12]:
johansen_result = coint_johansen(train_df, det_order=-1, k_ar_diff=5)
johansen_result

# VECM / VAR

$$\Delta y_t = \alpha \beta^T y_{t-1} + \Gamma_1 \Delta y_{t-1} + \dots + \Gamma_{p-1} \Delta y_{t-p+1} + u_t$$
where $\alpha, \beta \in \mathbb{R}^{K \times r}$ and $\Gamma_i \in \mathbb{R}^{K \times K}$ for $i = 1, \dots, p-1$ are the parameters and $u_t$ is $K$-dimensional white noise. Both $\alpha$ and $\beta$ have rank $r$ - then so called cointegration rank.

## Choose Deterministic Terms

In [13]:
deterministic = "colo" # "n", "ci", "co", "li", "lo"

## Select Lag Order

In [14]:
lag_order = select_order(data=train_df, maxlags=10, deterministic=deterministic)
lag_order.summary()

,AIC,BIC,FPE,HQIC
0,-1.552,-1.537*,0.2118,-1.547
1,-1.553,-1.532,0.2116,-1.545
2,-1.560,-1.532,0.2102,-1.550*
3,-1.560,-1.526,0.2102,-1.548
4,-1.560,-1.520,0.2102,-1.546
5,-1.560*,-1.514,0.2102*,-1.544
6,-1.559,-1.507,0.2103,-1.541
7,-1.557,-1.500,0.2107,-1.537
8,-1.557,-1.493,0.2108,-1.534
9,-1.556,-1.486,0.2110,-1.531


In [15]:
print(lag_order)

<statsmodels.tsa.vector_ar.var_model.LagOrderResults object. Selected orders are: AIC -> 5, BIC -> 0, FPE -> 5, HQIC ->  2>


In [16]:
lag_order.aic, lag_order.bic, lag_order.fpe, lag_order.hqic

(5, 0, 5, 2)

## Cointegration Rank

In [17]:
# `det_order=-1`: no deterministic terms
rank_test = select_coint_rank(train_df, det_order=-1, k_ar_diff=lag_order.aic, signif=0.05)
rank_test.rank

1

In [18]:
rank_test.summary()

r_0,r_1,test statistic,critical value
0,2,17.87,12.32
1,2,3.588,4.130


## Model

In [19]:
vecm = VECM(train_df, 
            k_ar_diff=lag_order.aic, 
            coint_rank=rank_test.rank,
            deterministic=deterministic)

In [20]:
vecm_res = vecm.fit()
vecm_res.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
const,-0.0555,0.038,-1.453,0.146,-0.130,0.019
lin_trend,5.737e-05,2.97e-05,1.933,0.053,-8.07e-07,0.000
L1.sp500,-0.0774,0.033,-2.318,0.020,-0.143,-0.012
L1.russell2000,0.0356,0.045,0.792,0.428,-0.052,0.124
L2.sp500,-0.1171,0.033,-3.509,0.000,-0.183,-0.052
L2.russell2000,0.0978,0.045,2.180,0.029,0.010,0.186
L3.sp500,-0.0140,0.033,-0.419,0.676,-0.080,0.052
L3.russell2000,0.0032,0.045,0.071,0.944,-0.085,0.091
L4.sp500,-0.0496,0.033,-1.487,0.137,-0.115,0.016
L4.russell2000,0.0367,0.045,0.821,0.412,-0.051,0.124


## Forecast

In [21]:
vecm_res.predict(steps=5)

array([[201.43788072, 124.92738811],
       [201.56076617, 124.97206802],
       [201.72530098, 125.04110499],
       [201.80567671, 125.07576766],
       [201.90113773, 125.11662653]])

In [22]:
for text, nd in zip(
    ["forecast", "lower", "upper"], 
    vecm_res.predict(steps=5, alpha=0.05) # 95% confidence level
):
    print(f"{text}:\n{nd}")

forecast:
[[201.43788072 124.92738811]
 [201.56076617 124.97206802]
 [201.72530098 125.04110499]
 [201.80567671 125.07576766]
 [201.90113773 125.11662653]]
lower:
[[199.18589196 123.2526217 ]
 [198.46152013 122.65689781]
 [198.02702691 122.23269254]
 [197.59964257 121.85179939]
 [197.26095074 121.54606716]]
upper:
[[203.68986947 126.60215452]
 [204.66001222 127.28723823]
 [205.42357504 127.84951744]
 [206.01171086 128.29973593]
 [206.54132472 128.6871859 ]]
